## 1 - Criar navegador e importar base de dados

In [137]:
# Importar modulos
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import smtplib
import email.message
import pandas as pd

# criar o navegador
nav = webdriver.Chrome()

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel(r"buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


## 2 - Definir funções para as partes do código que ocorrem mais de uma vez

In [138]:
# Analisa se o resultado não tem nenhum termo banido
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

# Analisa se o resultado tem todos os termos do nome do produto
def verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome):
    tem_todos_termos_produto = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produto = False
    return tem_todos_termos_produto


## 3 - Buscar elementos correspondentes aos parametros de busca no google Shopping

In [139]:
def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    
    # Tratar os valores
    produto = produto.lower()
    lista_termos_nome_produto = produto.split(" ")
    
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    
    lista_ofertas = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    # entrar no google
    nav.get("https://www.google.com")
    nav.find_element('xpath', '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)

    # Deixa o programa em stand by por 0.3s para conseguir acessar a aba shopping posteriormente
    time.sleep(0.3)

    # entrar aba shopping
    elementos = nav.find_elements("class name", 'hdtb-mitem')

    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break

    # pegar as informações dos produtos
    time.sleep(5)
    lista_resultados = nav.find_elements('class name', 'i0X6df')
    
    
    for resultado in lista_resultados:
        
        preco = resultado.find_element("class name", 'a8Pemb').text
        preco = preco.replace("US$", "").replace(" ", "").replace("R$","")
        nome = resultado.find_element("class name", 'tAxDx').text
        nome = nome.lower()
       

        # Analisar se o resultado não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        # Analisar se o resultado tem todos os termos do nome do produto
        tem_todos_termos_produto = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)

        # Selecionar apenas os elementos que tem_termos_banidos = False && tem_todos_termos_produto = True
        if not tem_termos_banidos and tem_todos_termos_produto:

            # Tratar o preço para transformar ele em float
            try:
                preco = preco.replace("R$","").replace(" ", "").replace(".", "").replace(",", ".").replace("/","").replace("mês","")
                preco = float(preco)
            
                
                # Analisa se o preco está entre preco_minimo e preco_maximo e busca o nome e o link desses resultados
                if preco_minimo <= preco <= preco_maximo:

                    elemento_referencia = resultado.find_element('class name', 'bONr3b')
                    elemento_pai = elemento_referencia.find_element('xpath','..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
            except:
                continue
        
                
    return lista_ofertas


## 4 - Buscar elementos correspondentes aos parametros de busca no Buscape

In [140]:
def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # Tratar os valores
    produto = produto.lower()
    lista_termos_nome_produto = produto.split(" ")
    
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    
    lista_ofertas = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    # Buscar no Buscape
    nav.get("https://www.buscape.com.br/")
    nav.find_element('xpath','//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    
    
    # Fica em stand by por 0.1 segundos repetidamente até enquanto não existe o elemento 'Select_Select__1S7HV'
    while len(nav.find_elements('class name', 'Select_Select__1S7HV')) < 1:
        time.sleep(0.1)
        
    # Pegar os resultados
    lista_resultados = nav.find_elements('class name', 'SearchCard_ProductCard_Inner__7JhKb')
    

    for resultado in lista_resultados:
        try:
            preco = resultado.find_element('class name', 'Text_MobileHeadingS__Zxam2').text
            nome = resultado.find_element('class name', 'Text_MobileLabelXs__ER_cD').text
            nome = nome.lower()
            link = resultado.get_attribute("href")


            # Analisar se o resultado não tem nenhum termo banido
            tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

            # Analisar se o resultado tem todos os termos do nome do produto
            tem_todos_termos_produto = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)


            # Selecionar apenas os elementos que tem_termos_banidos = False && tem_todos_termos_produto = True
            if not tem_termos_banidos and tem_todos_termos_produto:

                # Tratar o preço para transformar ele em float
                preco = preco.replace("R$","").replace(" ", "").replace(".", "").replace(",", ".").replace("/","").replace("mês","")
                preco = float(preco)



                # Analisar se o preço está entre o preco minimo e o preço maximo  
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome, preco, link))

        except:
            continue
        
        
    # Retornar a lista de ofertas do buscape
    return lista_ofertas

## 5 - Criar tabela de ofertas

In [141]:
# cria a tabela de ofertas que conterá todas as informações 
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    
    # pesquisar pelo produto com suas condições de pesquisa
    produto = tabela_produtos.loc[linha, 'Nome']
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]

    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    # Verifica se a lista possui elementos, se não -> não cria o DataFrame
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['Produto', 'Preco', 'Link'])
        # Adiciona a tabela_google_shopping a tabela de ofertas
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    # Verifica se a lista possui elementos, se não -> não cria o DataFrame
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['Produto', 'Preco', 'Link'])
        # Adiciona a tabela_buscape a tabela de ofertas
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None
        
        
display(tabela_ofertas)
    

,Produto,Preco,Link
0,iphone 12 64 gb branco vitrine 100% apple,3089.00,https://www.google.com/url?url=https://produto...
1,iphone 12 64gb branco novo,3299.99,https://www.google.com/url?url=http://loja.swi...
2,vitrine apple iphone 12 64gb original com gara...,3159.00,https://www.google.com/url?url=https://produto...
3,apple iphone 12 5g 64gb vitrine original garan...,3389.00,https://www.google.com/url?url=https://produto...
0,smartphone apple iphone 12 64gb câmera dupla,3127.00,https://www.buscape.com.br/celular/smartphone-...
0,placa de video rtx 3060 ti asus rog strix oc l...,4293.15,https://www.google.com/url?url=https://www.ama...
0,placa de video nvidia geforce rtx 3060 ti 8 gb...,4293.15,https://www.buscape.com.br/placa-de-video/plac...


## 6 - Exportar para o excel

## 7 - Enviar a tabela de ofertas pelo e-mail

In [142]:
if len(tabela_ofertas) > 0:
    corpo_email = f'''
    Prezados, bom dia!<br>
    <br>
    <p> Encontramos alguns produtos em oferta dentro da faixa de preço desejada, conforme a tabela abaixo<p> <br>
    <br>
    {tabela_ofertas.to_html(index = False)}<br>
    <br>
    Qualquer dúvida estou à disposição.<br>
    <br>
    Att.,<br>
    André Andriolli
    '''
    msg = email.message.Message()
    msg['Subject'] = "Ofertas do dia"
    msg['From'] = "pythonteste0001@gmail.com"
    msg['To'] = "pythonteste0001+Ofertas@gmail.com"
    password = 'nylucxlhankeqwxz'
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(corpo_email)


    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    s.login(msg['From'], password)
    s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
    print('E-mail enviado')

# Fechar o navegador ao enviar o email após a conclusão do código
nav.quit()

E-mail enviado
